## 1. 환경 설정

`(1) Env 환경변수`

In [1]:
#https://teddynote.com/10-RAG%EB%B9%84%EB%B2%95%EB%85%B8%ED%8A%B8/%ED%99%98%EA%B2%BD%20%EC%84%A4%EC%A0%95%20(Windows)/
from dotenv import load_dotenv
load_dotenv()

True

`(2) 라이브러리`

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. 도구 호출 (Tool Calling)
- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음 

### 2-1. 랭체인 내장 도구
- Tavily 웹 검색 도구 (예시)

`(1) 도구(tool) 정의하기`

In [3]:
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 설정
query = "양천구의 상권 규모와 아파트단지를 추천해주세요"

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearchResults(max_results=2)

# 웹 검색 실행
search_results = web_search.invoke(query)

# 검색 결과 출력
for result in search_results:
    print(result)  
    print("-" * 100)  

{'url': 'https://month.foodbank.co.kr/section/section_view.php?secIndex=1779&page=1&section=008012', 'content': '오목교역은 양천구 상권의 가장 핵심이다. 북쪽으로는 오피스 밀집지역 SBS 방송국과 목동아파트 1~7단지가, 남쪽으로는 양천구청, 관공서 및 목동아파트'}
----------------------------------------------------------------------------------------------------
{'url': 'https://www.youtube.com/watch?v=5uBLc8MmFFI', 'content': '#양천구 #목동 #신정동 #신월동 #목동신시가지 #목동아파트 #목동재건축 #신정뉴타운\n#호반써밋목동 #목동힐스테이트아파트 #래미안목동아델리체 #목동센트럴아이파크위브\n#도시 #도시분석 #도시리뷰 #도시리뷰어 #city #cityreview #cityreviewer\n#지역분석 #지역공부 #지역전망 #임장 #신도시 #재개발 #재건축 #택지개발 \n 11 comments [...] [CC] 서남권 대표적인 주거단지 양천구, 목동 신시가지를 비롯한 구석구석을 살펴보자! \n 도시리뷰어 | city reviewer \n 144 likes \n 8646 views \n 11 Mar 2023 \n 오늘은 서남권 대표적인 주거단지 양천구를 살펴볼께요.\n\n제가 마지막으로 올렸던 영상, 영등포구가 서남권의 대표적인 일자리를 책임지는 지역이라면, 양천구는 그보다는 한적하고 여유로운 주거단지의 느낌입니다.\n\n양천구하면 목동을 먼저 떠올리실텐데요.\n목동 외에도 신정동, 신월동 각각 1/3씩 면적을 차지하고 있습니다.\n그럼, 양천구의 이모저모 그리고 재건축, 재개발 소식을 총체적으로 살펴볼께요!'}
-----------------------------------------------------------------------

In [5]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema)
print("-"*100)

자료형: 
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search_results_json
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. Input '
 'should be a search query.')
----------------------------------------------------------------------------------------------------
schema: 
<class 'langchain_community.tools.tavily_search.tool.TavilyInput'>
----------------------------------------------------------------------------------------------------


`(2) 도구(tool) 호출하기`

In [6]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])

# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 82, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'finish_reason': 'stop', 'logprobs': None}, id='run--04fcdbf3-4002-475e-8da1-09857171fc4d-0', usage_metadata={'input_tokens': 82, 'output_tokens': 13, 'total_tokens': 95})
----------------------------------------------------------------------------------------------------
'안녕하세요! 무엇을 도와드릴까요?'
----------------------------------------------------------------------------------------------------
[]
----------------------------------------------------------------------------------------------------


In [7]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "20평대 아파트 평균가격이 가장 높은 구를 알려주세요"
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3BEY50dDQges5HzLXJYVURQE', 'function': {'arguments': '{"query":"20평대 아파트 평균가격 가장 높은 구"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 93, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--59b7da2d-3553-49c6-8916-7ade24d840ea-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '20평대 아파트 평균가격 가장 높은 구'}, 'id': 'call_3BEY50dDQges5HzLXJYVURQE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 93, 'output_tokens': 27, 'total_tokens': 120})
--------------------------------------

In [8]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'tavily_search_results_json',
 'args': {'query': '20평대 아파트 평균가격 가장 높은 구'},
 'id': 'call_3BEY50dDQges5HzLXJYVURQE',
 'type': 'tool_call'}

`(3) 도구(tool) 실행하기`

In [9]:
### 방법 1: 직접 도구 호출 처리

# 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
# 'args'를 사용하여 도구를 호출하고 결과를 얻는다.

tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)
print(tool_output)

tavily_search_results_json 호출 결과:
----------------------------------------------------------------------------------------------------
[{'url': 'https://www.newsspace.kr/news/article.html?no=6866', 'content': '특히 종로구는 2015년 3억5670만원에서 2025년 12억5887만원으로 253% 오르며 서울에서 가장 높은 상승률을 기록했다. 은평구, 강북구, 구로구, 노원구,'}, {'url': 'https://post.naver.com/viewer/postView.naver?volumeNo=37258600&memberNo=45336244', 'content': '강남4구 20평대 아파트 중 가격이 가장 낮게 거래된 2곳은? 마지막으로 동남권입니다. 강남4구라 불릴 정도로 서울에서 가장 높은 집값을 자랑하는 곳'}]


In [10]:
### 방법 2: ToolMessage 객체 생성

# 이 방법은 도구 호출 결과를 사용하여 ToolMessage 객체를 생성한다.
# 도구 호출의 ID와 이름을 포함하여 더 구조화된 메시지를 만든다.

from langchain_core.messages import ToolMessage
tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)

print(tool_message)

content=[{'url': 'https://www.newsspace.kr/news/article.html?no=6866', 'content': '특히 종로구는 2015년 3억5670만원에서 2025년 12억5887만원으로 253% 오르며 서울에서 가장 높은 상승률을 기록했다. 은평구, 강북구, 구로구, 노원구,'}, {'url': 'https://post.naver.com/viewer/postView.naver?volumeNo=37258600&memberNo=45336244', 'content': '강남4구 20평대 아파트 중 가격이 가장 낮게 거래된 2곳은? 마지막으로 동남권입니다. 강남4구라 불릴 정도로 서울에서 가장 높은 집값을 자랑하는 곳'}] name='tavily_search_results_json' tool_call_id='call_3BEY50dDQges5HzLXJYVURQE'


In [11]:
### 방법 3: 도구 직접 호출하여 바로 ToolMessage 객체 생성

# 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
# 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

tool_message = web_search.invoke(tool_call)

print(tool_message)


content='[{"url": "https://www.newsspace.kr/news/article.html?no=6866", "content": "특히 종로구는 2015년 3억5670만원에서 2025년 12억5887만원으로 253% 오르며 서울에서 가장 높은 상승률을 기록했다. 은평구, 강북구, 구로구, 노원구,"}, {"url": "https://post.naver.com/viewer/postView.naver?volumeNo=37258600&memberNo=45336244", "content": "강남4구 20평대 아파트 중 가격이 가장 낮게 거래된 2곳은? 마지막으로 동남권입니다. 강남4구라 불릴 정도로 서울에서 가장 높은 집값을 자랑하는 곳"}]' name='tavily_search_results_json' tool_call_id='call_3BEY50dDQges5HzLXJYVURQE' artifact={'query': '20평대 아파트 평균가격 가장 높은 구', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.newsspace.kr/news/article.html?no=6866', 'title': '[랭킹연구소] 서울시 20평대 아파트 10년간 최다상승, 강남 3구 아니네 ...', 'content': '특히 종로구는 2015년 3억5670만원에서 2025년 12억5887만원으로 253% 오르며 서울에서 가장 높은 상승률을 기록했다. 은평구, 강북구, 구로구, 노원구,', 'score': 0.8259413, 'raw_content': None}, {'url': 'https://post.naver.com/viewer/postView.naver?volumeNo=37258600&memberNo=45336244', 'title': '신혼부부 주목! 서울에서 제일 집값 낮은 20평대 아파트 10곳', 'content': '강남4구 20평대 아

In [12]:
pprint(tool_message.tool_call_id)
print()
pprint(tool_message.name)
print()
pprint(tool_message.content)
print()

'call_3BEY50dDQges5HzLXJYVURQE'

'tavily_search_results_json'

('[{"url": "https://www.newsspace.kr/news/article.html?no=6866", "content": '
 '"특히 종로구는 2015년 3억5670만원에서 2025년 12억5887만원으로 253% 오르며 서울에서 가장 높은 상승률을 기록했다. '
 '은평구, 강북구, 구로구, 노원구,"}, {"url": '
 '"https://post.naver.com/viewer/postView.naver?volumeNo=37258600&memberNo=45336244", '
 '"content": "강남4구 20평대 아파트 중 가격이 가장 낮게 거래된 2곳은? 마지막으로 동남권입니다. 강남4구라 불릴 정도로 '
 '서울에서 가장 높은 집값을 자랑하는 곳"}]')



In [13]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': '20평대 아파트 평균가격 가장 높은 구'},
  'id': 'call_3BEY50dDQges5HzLXJYVURQE',
  'type': 'tool_call'}]

In [14]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

[ToolMessage(content='[{"url": "https://www.newsspace.kr/news/article.html?no=6866", "content": "특히 종로구는 2015년 3억5670만원에서 2025년 12억5887만원으로 253% 오르며 서울에서 가장 높은 상승률을 기록했다. 은평구, 강북구, 구로구, 노원구,"}, {"url": "https://post.naver.com/viewer/postView.naver?volumeNo=37258600&memberNo=45336244", "content": "강남4구 20평대 아파트 중 가격이 가장 낮게 거래된 2곳은? 마지막으로 동남권입니다. 강남4구라 불릴 정도로 서울에서 가장 높은 집값을 자랑하는 곳"}]', name='tavily_search_results_json', tool_call_id='call_3BEY50dDQges5HzLXJYVURQE', artifact={'query': '20평대 아파트 평균가격 가장 높은 구', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.newsspace.kr/news/article.html?no=6866', 'title': '[랭킹연구소] 서울시 20평대 아파트 10년간 최다상승, 강남 3구 아니네 ...', 'content': '특히 종로구는 2015년 3억5670만원에서 2025년 12억5887만원으로 253% 오르며 서울에서 가장 높은 상승률을 기록했다. 은평구, 강북구, 구로구, 노원구,', 'score': 0.8259413, 'raw_content': None}, {'url': 'https://post.naver.com/viewer/postView.naver?volumeNo=37258600&memberNo=45336244', 'title': '신혼부부 주목! 서울에서 제일 집값 낮은 20평대 아파트 10곳', 'conten

`(4) ToolMessage를 LLM에 전달하여 답변을 생성하기`

In [15]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4.1-mini")

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[web_search])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)
    tool_msgs = web_search.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = web_search_chain.invoke("오늘 노량진 레미안 25평의 가격은 얼마인가요?")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_bDeKNSwFX5zZcwGcqWHT0UyK', 'function': {'arguments': '{"query":"노량진 레미안 25평 가격"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 114, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--fd5197bc-96ba-4b28-8041-fa0f5b4100bc-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '노량진 레미안 25평 가격'}, 'id': 'call_bDeKNSwFX5zZcwGcqWHT0UyK', 'type': 'tool_call'}] usage_metadata={'input_tokens': 114, 'output_tokens': 28, 'total_tokens': 142}
--------------------------------------------------------

### 2-2. 사용자 정의 도구
- @tool decorator를 통해 사용자 정의 도구를 정의할 수 있음

`(1) 도구(tool) 정의하기`

In [16]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

# Tool 정의 
@tool
def search_web(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [17]:
# 도구 속성
print("자료형: ")
print(type(search_web))
print("-"*100)

print("name: ")
print(search_web.name)
print("-"*100)

print("description: ")
pprint(search_web.description)
print("-"*100)

print("schema: ")
pprint(search_web.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_web
----------------------------------------------------------------------------------------------------
description: 
('Searches the internet for information that does not exist in the database or '
 'for the latest information.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Searches the internet for information that does not exist in '
                'the database or for the latest information.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_web',
 'type': 'object'}
----------------------------------------------------------------------------------------------------


In [18]:
query = "10억 대에 서울에 매매할 수 있는 아파트를 추천해주세요"
search_result = search_web.invoke(query)

print(search_result)

<Document href="https://m.blog.naver.com/jay_company/223139975988"/>
답십리동 장안동 휘경동 일대가

가성비가 꽤 괜찮은데요.

답십리 래미안위브 전용59 호가

답십리 파크자이 전용59 호가

답십리 래미안크레시티  전용59 호가

답십리 래미안미드카운티 전용59 호가

특히 답십리동 일대가

지난 조정장에서

하락 폭이

상당히 컸던 지역인데

​

답십리 내

4대장이라고 볼 수 있는

래미안위브, 파크자이,

래미안크레시티, 미드카운티까지

전용59의 경우

10억 이하의 금액대로

매수가 가능하겠습니다.

전농sk아파트 국평 호가

전농동sk아파트나

전농동우성아파트도

위 4대장에 비해

연식이 더 됐어도

더 큰 평형대로

상대적으로 저렴하게

매수가 가능하기 때문에

아이가 있으면서

가족 구성원 수가

어느정도 차있는 세대라면

이러한 단지들도

실거주하기에 괜찮은 곳이죠.

장안힐스테이트 전용59 호가

장안래미안2차 전용64 호가

또한

중랑천 일대가

상대적으로 깨끗하게

주거환경이 조성되는 편이기에 [...] 원리금 상환으로

고정 지출이 발생하게 됩니다.

​

10억은 더 증가되겠죠?

3억 내돈,

7억 대출받을시

400만원이 웃도는

원리금을

매달 갚아나가야 합니다.

​

이 부분을 꼭

먼저 인지하시고

매수에 임하셔야겠습니다.

동남권

10억이란 돈은

분명 작은 돈이 아니지만

강남권대에서

20평대 10억 이하의 매물을

찾긴 굉장히 어려운 편입니다.

​

특히 메인 중심부는

그러한 매물들이 아예 없습니다.

​

따라서

강남 외곽쪽으로

눈을 돌려야

10억 언저리 매물을

찾아볼 수 있는데요.

가락동

거여동

대표적으로

송파구 거여,풍납, 가락동

내에 있는 10억 선의

구축 20평대 아파트와

둔촌동

조금 더 외곽쪽으로 빠지게 되면

강동구 성내,둔촌,명일

구축 20평대 아파트를

7-8억대 정도로

찾아볼 수가 있겠습니다.

​

냉정하게

In [19]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[search_web])

# 도구 호출이 필요한 LLM 호출을 수행
query = "10억 이내로 서울에 매매할 아파트 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ypz8qLFJkh3XRZxk6c950fhR', 'function': {'arguments': '{"query":"서울 10억 이하 아파트 매매 추천"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 70, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b704874e-9072-47e7-a681-8bf8853a4bbc-0', tool_calls=[{'name': 'search_web', 'args': {'query': '서울 10억 이하 아파트 매매 추천'}, 'id': 'call_ypz8qLFJkh3XRZxk6c950fhR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 70, 'output_tokens': 23, 'total_tokens': 93})
----------------------------------------------------------------------------

`(2) LLM 도구 호출 성능 비교하기`

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

# 기본 LLM
llm_gemini_flash = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
llm_gemini_pro = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)


# LLM에 도구 바인딩하여 추가 
tools=[search_web]
gemini_flash_with_tools = llm_gemini_flash.bind_tools(tools)
gemini_pro_with_tools = llm_gemini_pro.bind_tools(tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


- gemini-1.5-flash

In [22]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "강남 상권의 특징을 알려주세요"
ai_msg = gemini_flash_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='강남 상권의 특징을 간략하게 설명하자면 다음과 같습니다.\n\n* **고소득층 집중:** 강남구는 전국에서 가장 높은 소득 수준을 가진 지역 중 하나이며, 이는 강남 상권의 주요 소비층이 고소득층임을 의미합니다.  고가의 상품과 서비스에 대한 수요가 높습니다.\n\n* **다양한 업종 밀집:**  고급 레스토랑, 백화점, 명품 매장, 엔터테인먼트 시설 등 다양한 업종의 고급 상업시설이 밀집되어 있습니다.  특히, 패션, 미용, 외식 분야가 발달했습니다.\n\n* **높은 임대료:**  높은 수요와 제한된 공급으로 인해 임대료가 매우 높습니다.  이는 상권 진입 장벽이 높다는 것을 의미합니다.\n\n* **경쟁 심화:**  많은 업체들이 강남 상권에 진출하려 하기 때문에 경쟁이 매우 치열합니다.  차별화된 상품과 서비스, 마케팅 전략이 중요합니다.\n\n* **트렌드 민감성:**  최신 유행에 민감하고, 새로운 트렌드를 빠르게 반영하는 특징이 있습니다.  소비자들의 니즈를 정확하게 파악하고 대응하는 것이 중요합니다.\n\n* **교통 편의성:** 지하철 및 버스 노선이 잘 발달되어 있어 접근성이 좋습니다.\n\n* **젊은층 유입:**  젊은 세대의 유입이 많아 최신 트렌드를 반영한 다양한 업종이 성장하고 있습니다.\n\n\n더 자세한 정보를 원하시면,  구체적으로 어떤 측면에 관심 있으신지 알려주세요. (예: 특정 업종, 연령대별 소비 특징, 상권 변화 추세 등)  제공된 정보만으로는 더 자세한 답변을 드리기 어렵습니다.  `default_api`는 웹 검색 기능을 제공하지 않으므로,  더 자세한 정보는 인터넷 검색을 통해 얻으실 수 있습니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='r

- gemini-1.5-pro

In [23]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "강남 상권의 특징을 알려주세요"
ai_msg = gemini_pro_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='강남 상권은 서울의 대표적인 번화가 중 하나로, 다음과 같은 특징을 가지고 있습니다.\n\n* **고급 상권**: 명품 브랜드, 고급 레스토랑, 백화점 등이 밀집해 있으며, 높은 소비 수준을 가진 고객층을 타겟으로 합니다.\n* **트렌드 중심**: 최신 유행과 트렌드를 선도하며, 패션, 뷰티, 엔터테인먼트 등 다양한 분야에서 새로운 트렌드를 접할 수 있습니다.\n* **유동 인구**: 지하철 2호선, 신분당선, 버스 등 대중교통이 잘 갖춰져 있어 유동 인구가 많습니다.\n* **24시간 활성화**: 밤늦게까지 영업하는 상점과 유흥 시설이 많아 밤 문화가 발달되어 있습니다.\n* **비즈니스 중심**: 대기업 본사, 금융기관, 무역회사 등이 밀집해 있어 비즈니스 활동이 활발하게 이루어집니다.\n* **외국인 관광객**: 쇼핑, 의료 관광 등을 위해 방문하는 외국인 관광객이 많습니다.\n* **높은 임대료**: 상권의 인기와 유동 인구로 인해 임대료가 매우 높은 편입니다.\n\n이러한 특징들을 바탕으로 강남 상권은 한국 경제와 문화의 중심지 역할을 하고 있습니다. 하지만 높은 임대료와 경쟁으로 인해 소규모 상점들은 어려움을 겪는 경우도 있습니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--fa0ad36b-cf8a-41c6-acb0-80db62af27b2-0', usage_metadata={'input_tokens': 57, 'output_tokens': 381, 'total_tokens': 438})
----------------------------------------------------------------------------------------------------
('

### 2-3. Runnable 객체를 도구(tool) 변환
- 문자열이나 dict 입력을 받는 Runnable을 도구로 변환
- as_tool 메서드를 사용

`(1) Document Loader`

In [24]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)

C:\Users\masta\AppData\Local\Temp\ipykernel_22696\2894064045.py:24: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_search = runnable.as_tool(


In [25]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_search
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a specified number of documents. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'k': {'default': 2,
                      'description': 'The number of documents to return '
                                     '(default is 2)',
                      'title': 'K',
                      'type': 'integer'},
                'q

In [28]:
# 위키 검색 실행
query = "강남상권의 특징"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(result)  
    print("-" * 100)  

page_content='서초 삼성타운(영어: Samsung Town Headquarters Seocho)은 대한민국 서울특별시 서초구 서초동에 있는 삼성그룹의 핵심 계열사들이 모여 있는 마천루 오피스 단지이다.
건물 연면적은 110,800m²이며, 상주인원은 2만명에 달한다고 한다.


== 역사 ==


=== 계획 ===
삼성그룹을 쩔쩔매게 한 비범한 건물의 이름은 '윤빌딩'. 본래 건물주의 성을 따서 지은 이름이다. 삼성타운 건립 계획이 기획된 것은 지난 86년부터다. 당시 삼성생명은 패션단지 건립을 위해 A동 사업부지를 매입했다. 이후 93년부터는 본격적으로 인근 부지 추가매입이 시작됐다. 95년에는 그룹차원에서 S-프로젝트를 추진키로 결정했다. 당시 프로젝트에 참여했던 한 관계자는 "이후 수차례의 설계변경이 있었지만 뼈 대가 되는 개발구상과 마스터플랜은 초기 단계의 기획안이 대체로 유지되는 수 준이었다"고 말했다. S-프로젝트를 추진한 태스크포스팀은 당초 이 단지를 패션 및 영상단지를 포함하는 복합단지로 조성할 계획이었다.
사실 삼성그룹이 복합 업무단지로서의 본사를 설립하려는 계획은 꽤 예전인 1995년 무렵부터 시작되었다. 삼성그룹 계열사가 점차 늘어나고 있었는데, 당초 중구 태평로의 삼성본관에 모두 입주할 여력이 없어서 이래저래 계열사가 흩어져 있었기 때문이다.
이에 삼성그룹은 1995년 무렵, 도곡동에 위치한 구 공군 사격장 부지를 검토하기 시작한다. 이 곳을 인수하여, 지상 102층, 380m 정도의 마천루를 건립하여 복합 삼성타운을 조성하기로 한 것이다. 이에 따라 1996년 이 부지를 인수했다. 그러나 계획이 아직도 있었다.
1999년에는 100층 미만으로 추진하고 2002년에는 이 부지에 신라호텔이 특급 호텔 건립을 검토하고 있다고 밝히기도 했지만 곧 백지화됐다. 2003년에는 삼성타운의 계획이 아직도 있었다.


=== 개발 ===
한편, 업무지구로서의 삼성타운은 강남역 부근 초알짜배기 땅에 마천루 세 동을 지음으로서 대체하기로 했는데

In [29]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 상권 특징은 무엇인가요? 그리고 유동인구를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5Xx5aVtuq2g98kdHQGaTDfhl', 'function': {'arguments': '{"query": "서울 강남 상권 특징"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_5iJdN1qMtzkvhiXmRws6nK7K', 'function': {'arguments': '{"query": "서울 강남 유동인구"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 170, 'total_tokens': 225, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_38647f5e19', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--add06aac-2150-49d2-a0a4-4698faab40bf-0', tool_calls=[{'name': 'search_web', 'args': {'query': '서울 강남 상권 특징'}, 'id': 'call_5Xx5aVtuq2g98kdHQGaTDfhl', 'type': 'tool_call'}, {'name': 'search_web', 'args': 

`(2) LCEL 체인`
- 위키피디아 문서를 검색하고 내용을 요약하는 체인

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"송파 상권의 특징"})
pprint(summarized_text)

('The Gyeongui Line Forest Park is a linear park in South Korea, covering '
 'approximately 101,668 m² and extending 6.3 km. It was created from the land '
 'left after the underground construction of the Yongsan Line, with '
 'development starting in 2009 and completion in May 2016. The park features '
 'sections from Hongjecheon to the Yongsan Cultural and Sports Center and is '
 "known for its resemblance to New York's Central Park, particularly the area "
 'near Hongdae. Managed by a non-profit organization, the park aims to enhance '
 'local community engagement and ecological infrastructure.\n'
 '\n'
 'Lotte World Tower, located in Seoul, is a skyscraper that stands 555 meters '
 'tall with 123 floors. Construction began in 2010, and it was completed in '
 'December 2016, officially opening in April 2017. Initially proposed as a '
 'second Lotte World in 1994, the project evolved through various plans and '
 'name changes. It is the tallest building in South Korea and the sixth '

In [31]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema)
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
<class '__main__.WikiSummarySchema'>
----------------------------------------------------------------------------------------------------


In [40]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "강남의 파스타 맛집은 어디인가요? 이유를 설명해주세요 "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aWD3ja93BHldTyqBG4dw8PMo', 'function': {'arguments': '{"query":"강남 파스타 맛집 추천 2023"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 141, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e7320eb6-096d-4cbe-90dc-13075e788c0f-0', tool_calls=[{'name': 'search_web', 'args': {'query': '강남 파스타 맛집 추천 2023'}, 'id': 'call_aWD3ja93BHldTyqBG4dw8PMo', 'type': 'tool_call'}], usage_metadata={'input_tokens': 141, 'output_tokens': 24, 'total_tokens': 165})
-----------------------------------------------------------------------------

In [43]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)
    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_RmzUTgwuGOZjyvdS2ruIJ66O', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 120, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--fefd6c23-57d5-45fc-acf1-8de121c60eb3-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_RmzUTgwuGOZjyvdS2ruIJ66O', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 20, 'total_tokens': 140}
---------------------------------------------------------------------------------------------------

### 2-4. 벡터저장소 검색기
- @tool decorator 사용

`(1) 문서 로드 및 인덱싱`

In [73]:
from langchain.document_loaders import TextLoader

# 구판 텍스트 데이터를 로드
loader = TextLoader("../data/Apartment.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [74]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_gu_items(document):
    """
    구 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    gu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 구 항목을 Document 객체로 변환
    gu_documents = []
    for i, item in enumerate(gu_items, 1):
        # 구 이름 추출
        gu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        gu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "gu_number": i,
                "gu_name": gu_name
            }
        )
        gu_documents.append(gu_doc)
    
    return gu_documents


# 구 항목 분리 실행
gu_documents = []
for doc in documents:
    gu_documents += split_gu_items(doc)

# 결과 출력
print(f"총 {len(gu_documents)}개의 구 항목이 처리되었습니다.")
for doc in gu_documents[:2]:
    print(f"\n구 번호: {doc.metadata['gu_number']}")
    print(f"구 이름: {doc.metadata['gu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 25개의 구 항목이 처리되었습니다.

구 번호: 1
구 이름: 서초구
내용:
1. 서초구
   • 20평대 평균가: 21.7억
   • 주요 아파트: 반포자이 3410세대, 래미안퍼스티지 2444세대, 아크로리버파크 1612세대
   • 설명: 한강변, 명...

구 번호: 2
구 이름: 강남구
내용:
2. 강남구
   • 20평대 평균가: 20.5억
   • 주요 아파트: 현대캐피탈 1520세대, 타워팰리스 1010세대, 삼성래미안 1580세대
   • 설명: 비즈니스 중심지,...


In [94]:
# Chroma Vectorstore를 사용하기 위한 준비
from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Chroma 인덱스 생성
gu_db = Chroma.from_documents(
    documents=gu_documents, 
    embedding=embeddings_model,   
    collection_name="apartment",
    persist_directory="./chroma_db",
)

# Retriever 생성
gu_retriever = gu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "평균값이 가장 비싼 구는 어디인가요? 특징은 무엇인가요?"
docs = gu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"구 번호: {doc.metadata['gu_number']}")
    print(f"구 이름: {doc.metadata['gu_name']}")
    print()

검색 결과: 2개
구 번호: 20
구 이름: 구로구

구 번호: 20
구 이름: 구로구



- 상권에 대해서도 같은 작업을 처리

In [93]:
# 상권 텍스트 데이터를 로드
loader = TextLoader("../data/Commercial.txt", encoding="utf-8")
documents = loader.load()

# 구 항목 분리 실행
gu_documents = []
for doc in documents:
    gu_documents += split_gu_items(doc)

# 결과 출력
print(f"총 {len(gu_documents)}개의 구 항목이 처리되었습니다.")
for doc in gu_documents[:2]:
    print(f"\n구 번호: {doc.metadata['gu_number']}")
    print(f"구 이름: {doc.metadata['gu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=gu_documents, 
    embedding=embeddings_model,   
    collection_name="Commercial",
    persist_directory="./chroma_db",
)

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "두번째로 GDP가 높은 구의 정보를 출력해주세요"
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"구 번호: {doc.metadata['gu_number']}")
    print(f"구 이름: {doc.metadata['gu_name']}")
    print()

총 25개의 구 항목이 처리되었습니다.

구 번호: 1
구 이름: 강남구
내용:
1. 강남구
   • 인당 GDP: 7,500만원
   • 주요 기업: 삼성전자, 현대자동차, 카카오
   • 상권 설명: 대형 오피스, 고급 상업시설, 유동인구 많음
   • 유...

구 번호: 2
구 이름: 중구
내용:
2. 중구
   • 인당 GDP: 6,800만원
   • 주요 기업: 신세계, 롯데, 한화
   • 상권 설명: 명동, 남대문, 을지로 등 중심상권, 관광객·직장인 유동인구
   ...
검색 결과: 2개
구 번호: 10
구 이름: 강서구

구 번호: 10
구 이름: 강서구



`(2) 도구(tool) 정의하기`

In [98]:
# 벡터 저장소 로드
gu_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="apartment",
    persist_directory="./chroma_db",
)

@tool
def search_apartment(query: str) -> List[Document]:
   """
   아파트(apartment) 정보를 안전하게 검색하는 도구입니다.
   이 도구는 아파트 관련 질의에만 사용하세요.
   """
   docs = gu_db.similarity_search(query, k=2)
   if len(docs) > 0: 
     return docs
   return [Document(page_content="관련 구 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(search_apartment))
print("-"*100)

print("name: ")
print(search_apartment.name)
print("-"*100)

print("description: ")
pprint(search_apartment.description)
print("-"*100)

print("schema: ")
pprint(search_apartment.args_schema)
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_apartment
----------------------------------------------------------------------------------------------------
description: 
'아파트(apartment) 정보를 안전하게 검색하는 도구입니다.\n이 도구는 아파트 관련 질의에만 사용하세요.'
----------------------------------------------------------------------------------------------------
schema: 
<class 'langchain_core.utils.pydantic.search_apartment'>
----------------------------------------------------------------------------------------------------


In [105]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# 벡터 저장소 로드
commercial_db = Chroma(
   embedding_function=embeddings_model,   
   collection_name="commercial",
   persist_directory="./chroma_db",
)

@tool
def search_commercial(query: str) -> List[Document]:
   """
   상권(Commercial) 정보를 안전하게 검색하는 도구입니다.
   이 도구는 상권 및 GDP 관련 질의에만 사용하세요.
   """
   docs = commercial_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 상권 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(search_commercial))
print("-"*100)

print("name: ")
print(search_commercial.name)
print("-"*100)

print("description: ")
pprint(search_commercial.description)
print("-"*100)

print("schema: ")
pprint(search_commercial.args_schema)
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_commercial
----------------------------------------------------------------------------------------------------
description: 
'상권(Commercial) 정보를 안전하게 검색하는 도구입니다.\n이 도구는 상권 및 GDP 관련 질의에만 사용하세요.'
----------------------------------------------------------------------------------------------------
schema: 
<class 'langchain_core.utils.pydantic.search_commercial'>
----------------------------------------------------------------------------------------------------


In [101]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
# LLM에 도구를 바인딩 (아파트/상권 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[search_apartment, search_commercial])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서초구의 20평대 아파트 평균가와 특징은 무엇인가요? 그리고 서초구 상권의 주요 특징도 알려주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g9KyHDLiLR5wP8hSty6tx2zT', 'function': {'arguments': '{"query": "서초구 20평대 아파트 평균가"}', 'name': 'search_apartment'}, 'type': 'function'}, {'id': 'call_7rDqS9h0nQcXxnsTgvSJ65IR', 'function': {'arguments': '{"query": "서초구 상권 주요 특징"}', 'name': 'search_commercial'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 141, 'total_tokens': 204, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7406c5ea-8c6b-4324-945e-ed2c8dee7803-0', tool_calls=[{'name': 'search_apartment', 'args': {'query': '서초구 20평대 아파트 평균가'}, 'id': 'call_g9KyHDLiLR5wP8hSty6tx2zT', 'type': 'tool_call'}, 

`(3) 여러 개의 도구(tool) 호출하기`

In [102]:
tools = [search_web, wiki_summary, search_apartment, search_commercial]
for tool in tools:
    print(tool.name)

search_web
wiki_summary
search_apartment
search_commercial


In [103]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4.1-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def gu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_apartment":
            tool_message = search_apartment.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_commercial":
            tool_message = search_commercial.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = gu_chain.invoke("강동구 아파트의 평균가격은 얼마인가요? 상권의 특성과 GDP도 알려주세요")

# 응답 출력 
print(response.content)

search_apartment: 
{'name': 'search_apartment', 'args': {'query': '강동구 아파트 평균 가격'}, 'id': 'call_0eoV52cTZwX38FMwV7vRYSyK', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_commercial: 
{'name': 'search_commercial', 'args': {'query': '강동구 상권 특성'}, 'id': 'call_DFhNhmIfLJMY12DXLJmkozrs', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_web: 
{'name': 'search_web', 'args': {'query': '강동구 GDP'}, 'id': 'call_d0MSzRXVVS2iHverKfXcMgMZ', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'gu_name': '강동구', 'gu_number': 11, 'source': '../data/Apartment.txt'}, page_content='11. 강동구\\n   • 20평대 평균가: 10.2억\\n   • 주요 아파트: 천호동 한라 1500세대, 고덕 롯데 1600세대, 성내 현대 1400세대\\n   • 설명: 도심에서 떨어져 있지만 자연환경과 주거환경이 좋은 지역.'), Document(me

In [110]:
# 체인 실행
response = gu_chain.invoke("송파구 아파트 가격과 GDP를 알려주세요")

# 응답 출력 
print(response.content)

search_apartment: 
{'name': 'search_apartment', 'args': {'query': '송파구 아파트 가격'}, 'id': 'call_sv5w8JLClMbhoNvmSkZ7FvaY', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_web: 
{'name': 'search_web', 'args': {'query': '송파구 GDP'}, 'id': 'call_hxbALpjBGwKtM7MlqnyQhhXc', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'gu_name': '송파구', 'gu_number': 3, 'source': '../data/Apartment.txt'}, page_content='3. 송파구\\n   • 20평대 평균가: 17.0억\\n   • 주요 아파트: 잠실엘스 5678세대, 리센츠 5563세대, 헬리오시티 9510세대\\n   • 설명: 대단지 신축, 한강·공원 인접, 학군과 생활 인프라 우수.'), Document(metadata={'gu_name': '송파구', 'gu_number': 3, 'source': '../data/Apartment.txt'}, page_content='3. 송파구\\n   • 20평대 평균가: 17.0억\\n   • 주요 아파트: 잠실엘스 5678세대, 리센츠 5563세대, 헬리오시티 9510세대\\n   • 설명: 대단지 신축, 한강·공원 인접, 학군과 생활 인프라 우수.')]", name='search_apartment', to

## 3. Few-shot 프롬프팅 
- 각 도구의 용도를 구분하여 few-shot 예제로 제시

### 3-1. Few-shot 도구 호출

In [111]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("서초구 20평대 아파트 평균가와 특징, 그리고 어울리는 상권에 대해 알려주세요.", name="example_user"),
    AIMessage("서초구 아파트 정보를 검색하고, 추가로 서초구 상권 정보를 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_apartment", "args": {"query": "서초구 20평대 아파트"}, "id": "1"}]),
    ToolMessage("서초구 20평대 평균가: 21.7억, 주요 아파트: 반포자이, 래미안퍼스티지, 아크로리버파크. 한강변, 명문학군, 대규모 재건축 단지로 고급화된 주거지.", tool_call_id="1"),
    AIMessage("서초구 20평대 아파트의 평균가는 21.7억이며, 반포자이, 래미안퍼스티지, 아크로리버파크 등이 대표 단지입니다. 한강변, 명문학군, 대규모 재건축 단지로 고급화된 주거지라는 특징이 있습니다. 이제 어울리는 상권 정보를 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_commercial", "args": {"query": "서초구 상권"}, "id": "2"}]),
    ToolMessage("서초구 인당 GDP: 5,700만원, 주요 기업: 현대모비스, 예술의전당. 법조타운, 예술·문화 중심지, 고급 주거·상업 복합. 유동인구: 800,000명.", tool_call_id="2"),
    AIMessage("서초구 상권은 인당 GDP 5,700만원, 현대모비스와 예술의전당 등 주요 기업이 위치해 있습니다. 법조타운, 예술·문화 중심지, 고급 주거·상업 복합이 특징이며, 유동인구는 약 80만 명입니다.", name="example_assistant"),
    AIMessage("서초구 20평대 아파트와 상권 정보를 종합하면, 고급 주거지와 다양한 문화·상업 인프라가 결합된 지역임을 알 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing Seoul apartment and commercial district information.
For information about apartments, use the search_apartment tool.
For commercial district information, use the search_commercial tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[search_apartment, search_commercial])

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

query = "강남구 20평대 아파트 평균가와 대표 단지, 그리고 강남구 상권의 주요 특징을 알려주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'search_apartment', 'args': {'query': '강남구 20평대 아파트'}, 'id': 'call_7kSVE1843rCjzPUUb5olL7mB', 'type': 'tool_call'}
{'name': 'search_commercial', 'args': {'query': '강남구 상권'}, 'id': 'call_Axu0ignvCrQ5X9EBUavSgEeh', 'type': 'tool_call'}


In [112]:
# 체인 실행
query = "양천구의 인구 및 GDP를 알려주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'search_commercial', 'args': {'query': '양천구 인구 GDP'}, 'id': 'call_gzzBRTZZeM5QXU5J4bpBpMqV', 'type': 'tool_call'}


### 3-2. 답변 생성 체인 

In [114]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 

system = """You are an AI assistant providing Seoul apartment and commercial district information.
For information about apartments, use the search_apartment tool.
For commercial district information, use the search_commercial tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4.1-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def gu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_commercial":
            tool_message = search_commercial.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_apartment":
            tool_message = search_apartment.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "은평구의 GDP와 평균집값을 알려주세요"
response = gu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

search_web: 
{'name': 'search_web', 'args': {'query': '은평구 GDP'}, 'id': 'call_xkkT0bJxCse1cFJrvZT4JRIJ', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_apartment: 
{'name': 'search_apartment', 'args': {'query': '은평구 평균 집값'}, 'id': 'call_Jmo0UQJVRy2BKQSQ2Eq7j52I', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content='<Document href="https://www.epnews.net/news/articleView.html?idxno=21547"/>\n기업이 적고 대표적인 베드타운인 은평구는 지역내총생산 규모가 4조 7540억 원으로 서울에서 22위로 낮은 편에 속했다. 지역내총생산 규모도 작은데\n</Document>\n---\n<Document href="https://www.epnews.net/news/articleView.html?idxno=33526"/>\n자치구별 1인당 GRDP규모는 중구 4억 8140만원, 종로구 2억 3860만원, 강남구 1억 5536만원 순으로 컸음. · 은평구는 1114만원으로 가장 작았음.\n</Document>', name='search_web', tool_call_id='call_xkkT0bJxCse1cFJrvZT4JRIJ'), ToolMessage(content="[Document(metadata={'gu_name': '은평구', 'gu_numbe

In [115]:
# 체인 실행
query = "양천구에 대해 알려주세요"
response = gu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '양천구'}, 'id': 'call_RlnDQVvdnKZVpGZMCy7qOsyR', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content='양천구는 서울특별시 남서부에 위치한 구로, 1988년 강서구에서 분리되었다. 북쪽은 강서구, 서쪽은 부천시, 남쪽은 구로구, 동쪽은 영등포구와 접한다. 양천구는 역사적으로 고려 시대부터 이름이 사용되었으며, 현재 18개의 행정동으로 나뉘어 있다. 인구는 2016년 기준 477,739명으로, 인구 밀도가 대한민국 기초자치단체 중 가장 높다. 주거 지역이 70% 이상을 차지하며, 목동 대단위 주택 단지가 유명하다. 주요 산업으로는 방송 및 IT 관련 기업이 있으며, 교통은 지하철과 도로망이 발달해 있다.\n\n영등포구는 서울특별시 남서부에 위치하며, 면적은 24.37km²로 16만 세대가 거주한다. 한강과 안양천이 합류하는 지점에 위치하고, 서울과 수도권 서남부를 연결하는 중심지 역할을 한다. 영등포구는 1936년 경성부에 편입되었으며, 현재 34개 동으로 구성되어 있다. 주요 시설로는 여의도와 국회의사당, 다양한 상업시설이 있으며, 교통은 경부선과 여러 지하철 노선이 연결되어 있다.', name='wiki_summary', tool_call_id='call_RlnDQVvdnKZVpGZMCy7qOsyR')]
----------------------------------------------------------------------------------------------------
('양천구는 서울특별시 남서부에 위치한 구로, 1988년에 강서구에서 분리되었습니다. 북쪽은 강서구, 서쪽은 부천시, 남

## 4. LangChain Agent 사용
- 유의사항: 프롬프트에 "agent_scratchpad",  "input" 변수를 포함

In [116]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing Seoul apartment and commercial district information. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For apartment information, use the search_apartment tool. This tool provides details on each district's 20평대 아파트 평균가, 대표 단지(세대수), 설명(학군, 환경 등)을 제공합니다.
        2. For commercial district information, use the search_commercial tool. This tool provides 인당 GDP, 주요 기업, 상권 설명, 유동인구 등 상권의 핵심 정보를 제공합니다.
        3. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        4. Provide clear and concise responses based on the search results.
        5. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        6. Always maintain a helpful and professional tone.
        7. When providing apartment information, describe in the order of 평균가, 대표 단지, 설명.
        8. When providing commercial information, describe in the order of 인당 GDP, 주요 기업, 상권 설명, 유동인구.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.

        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


In [117]:
# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [search_web, wiki_summary, search_commercial , search_apartment]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [118]:
# AgentExecutor 실행

query = "GDP 대비 평균 아파트가격이 낮고, 매매를 추천할만한 구는 어디인가요? "
agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `search_commercial` with `{'query': '서울 구별 인당 GDP'}`


[Document(metadata={}, page_content='관련 상권 정보를 찾을 수 없습니다.')]
Invoking: `search_apartment` with `{'query': '서울 구별 20평대 아파트 평균가'}`


[Document(metadata={'gu_name': '구로구', 'gu_number': 20, 'source': '../data/Apartment.txt'}, page_content='20. 구로구\n   • 20평대 평균가: 6.8억\n   • 주요 아파트: 신도림 삼성 1200세대, 구로 현대 1300세대\n   • 설명: 산업단지가 많아 근로자들이 거주하는 비율이 높음.'), Document(metadata={'gu_name': '구로구', 'gu_number': 20, 'source': '../data/Apartment.txt'}, page_content='20. 구로구\n   • 20평대 평균가: 6.8억\n   • 주요 아파트: 신도림 삼성 1200세대, 구로 현대 1300세대\n   • 설명: 산업단지가 많아 근로자들이 거주하는 비율이 높음.')]현재 서울 구별 인당 GDP 정보는 찾을 수 없으나, 구로구 아파트 정보는 다음과 같습니다.

- 20평대 아파트 평균가: 6.8억 원
- 대표 단지: 신도림 삼성 1200세대, 구로 현대 1300세대
- 설명: 산업단지가 많아 근로자들이 거주하는 비중이 큽니다.

GDP 대비 아파트 가격이 낮고 매매를 추천할 만한 구를 정확히 판단하려면 인당 GDP 데이터가 필요합니다. 인당 GDP 데이터를 추가로 확인하거나, 특정 구에 대해 더 알고 싶으시면 알려주세요.

> Finished chain.


In [119]:
pprint(agent_response)

{'input': 'GDP 대비 평균 아파트가격이 낮고, 매매를 추천할만한 구는 어디인가요? ',
 'output': '현재 서울 구별 인당 GDP 정보는 찾을 수 없으나, 구로구 아파트 정보는 다음과 같습니다.\n'
           '\n'
           '- 20평대 아파트 평균가: 6.8억 원\n'
           '- 대표 단지: 신도림 삼성 1200세대, 구로 현대 1300세대\n'
           '- 설명: 산업단지가 많아 근로자들이 거주하는 비중이 큽니다.\n'
           '\n'
           'GDP 대비 아파트 가격이 낮고 매매를 추천할 만한 구를 정확히 판단하려면 인당 GDP 데이터가 필요합니다. 인당 '
           'GDP 데이터를 추가로 확인하거나, 특정 구에 대해 더 알고 싶으시면 알려주세요.'}


## 5. Gradio 활용

In [120]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-2:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "서초구 20평대 아파트 평균가와 대표 단지를 알려주세요.",
    "강남구 상권의 주요 특징과 유동인구는 얼마인가요?",
    "송파구에서 학군이 좋은 아파트 단지를 추천해 주세요.",
    "마포구 20평대 아파트와 상권 정보를 비교해 주세요."
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="서울 아파트·상권 AI 어시스턴트",
    description="서울 25개구의 20평대 아파트, 대표 단지, 상권 정보, 유동인구 등 다양한 부동산·상권 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# 데모 실행
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...

Invoking: `search_apartment` with `{'query': '서초구 20평대 아파트'}`


[Document(metadata={'gu_name': '서초구', 'gu_number': 1, 'source': '../data/Apartment.txt'}, page_content='1. 서초구\n   • 20평대 평균가: 21.7억\n   • 주요 아파트: 반포자이 3410세대, 래미안퍼스티지 2444세대, 아크로리버파크 1612세대\n   • 설명: 한강변, 명문학군, 대규모 재건축 단지로 고급화된 주거지.'), Document(metadata={'gu_name': '서초구', 'gu_number': 1, 'source': '../data/Apartment.txt'}, page_content='1. 서초구\n   • 20평대 평균가: 21.7억\n   • 주요 아파트: 반포자이 3410세대, 래미안퍼스티지 2444세대, 아크로리버파크 1612세대\n   • 설명: 한강변, 명문학군, 대규모 재건축 단지로 고급화된 주거지.')]서초구 20평대 아파트 평균가는 약 21.7억 원입니다. 대표 단지로는 반포자이(3410세대), 래미안퍼스티지(2444세대), 아크로리버파크(1612세대)가 있습니다. 이 지역은 한강변에 위치해 있고 명문학군과 대규모 재건축 단지로 고급화된 주거지로 알려져 있습니다. 도움이 필요하시면 언제든 말씀해 주세요!

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `search_commercial` with `{'query': '강남구 상권'}`


[Document(metadata={}, page_content='관련 상권 정보를 찾을 수 없습니다.')]현재 강남구 상권의 구체적인 주요 특징과 유동인구에 대한 정보가 바로 확인되지 않습니다. 조금 더 구체적인 상권 이름이나 지역을

In [121]:
# 데모 종료
demo.close()

Closing server running on port: 7860
